In [1]:
import os
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
plt.style.use('fivethirtyeight')
plt.rcParams['figure.dpi']= 180

In [9]:
file_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'job_ads.csv'))
ads = pd.read_csv(file_path, index_col=0)
ads.drop_duplicates(inplace=True)

In [10]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 611 entries, 0 to 106
Data columns (total 7 columns):
id                  611 non-null object
job_descr           490 non-null object
location            606 non-null object
minimum_qual        490 non-null object
preferred_qual      490 non-null object
responsibilities    490 non-null object
title               611 non-null object
dtypes: object(7)
memory usage: 38.2+ KB
